In [2]:
! pip install WordNetLemmatizer

ERROR: Could not find a version that satisfies the requirement WordNetLemmatizer (from versions: none)
ERROR: No matching distribution found for WordNetLemmatizer

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np

In [4]:
columns = ['id', 'contract_number', 'object_name', 'object_code', 'cost', 'contract_execution_days']
dtypes = {'id': lambda x: pd.to_numeric(x, errors="coerce"), 
          'contract_number': str, 
          'object_name': str,
          'object_code': str, 
          'cost':lambda x: pd.to_numeric(x, errors="coerce"), 
          'contract_execution_days': lambda x: pd.to_numeric(x, errors="coerce")}

In [5]:
df = pd.read_csv('datasets/dataset_filter.csv', sep=';', low_memory=False, on_bad_lines='skip')
df.shape

(900153, 6)

In [6]:
# Получим стоп слова русского языка
import nltk
#nltk.download()
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\head\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\head\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\head\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\head\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\head\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from nltk.corpus import stopwords
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [8]:
# получим знаки пунктуации
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [9]:
# Создаём список шума из стопслов и знаков пунктуации
noise = stopwords.words('russian') + list(punctuation)

In [11]:
df.head()

,id,contract_number,object_name,object_code,cost,contract_execution_days,data
0,137439,2590407717221000153,Работы строительные специализированные,43.9,539265.60,29,"[работы, строительные, специализированные]"
1,137528,2590421285821000134,Работы строительные специализированные,43.9,2299469.98,122,"[работы, строительные, специализированные]"
2,137529,2590421285821000135,Работы строительные специализированные,43.9,1898111.58,122,"[работы, строительные, специализированные]"
3,137827,2590500335021000676,Работы строительные специализированные,43.3,341914.00,199,"[работы, строительные, специализированные]"
4,138113,2590615606021000060,Услуги в области архитектуры и инженерно-техни...,71.1,1714352.37,383,"[услуги, в, области, архитектуры, и, инженерно..."


In [56]:
from nltk import download
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict, Counter
from nltk.corpus import wordnet as wn  # Проверить необходимость


tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

stemmer = nltk.stem.snowball.RussianStemmer()
word_lemmatized = WordNetLemmatizer()

for index, entry in zip(df["data"].index, df["data"]):
    # Declaring Empty List to store the words that follow the rules for this step
    final_words = []
    # Initializing WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words("russian") and word.isalpha():
            word_final = word_lemmatized.lemmatize(word, tag_map[tag[0]])
            word_final = stemmer.stem(word_final)
            final_words.append(word_final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index, "data_final"] = str(final_words)
    
df["data_final"]

0                 ['работ', 'строительн', 'специализирова']
1                 ['работ', 'строительн', 'специализирова']
2                 ['работ', 'строительн', 'специализирова']
3                 ['работ', 'строительн', 'специализирова']
4         ['услуг', 'област', 'архитектур', 'проектирова...
                                ...                        
551495            ['работ', 'строительн', 'специализирова']
551496            ['работ', 'строительн', 'специализирова']
551497            ['работ', 'строительн', 'специализирова']
551498    ['услуг', 'област', 'архитектур', 'проектирова...
551499                ['здан', 'работ', 'возведен', 'здан']
Name: data_final, Length: 551500, dtype: object

In [55]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

data = df['object_name'].values.astype('U')

from nltk.tokenize import sent_tokenize, word_tokenize

#df["data"] = [word_tokenize(entry.lower()) for entry in df["object_name"]]

# получаем список токенов
df['tokens'] = [word_tokenize(text.lower(), language="russian") for text in data]

# лематизируем полученные токены
lemma = WordNetLemmatizer()
df['lemma'] = [[lemma.lemmatize(token) for token in text] for text in df['tokens']]
#df['lemma'] = lemma.lemmatize(df['tokens'].values)

# Очищаем от шума
df['final']=[[token for token in text if token not in noise] for text in df['lemma']]

df.head()

,id,contract_number,object_name,object_code,cost,contract_execution_days,data,data_final,tokens,lemma,final
0,137439,2590407717221000153,Работы строительные специализированные,43.9,539265.60,29,"[работы, строительные, специализированные]","['работ', 'строительн', 'специализирова']","[работы, строительные, специализированные]","[работы, строительные, специализированные]","[работы, строительные, специализированные]"
1,137528,2590421285821000134,Работы строительные специализированные,43.9,2299469.98,122,"[работы, строительные, специализированные]","['работ', 'строительн', 'специализирова']","[работы, строительные, специализированные]","[работы, строительные, специализированные]","[работы, строительные, специализированные]"
2,137529,2590421285821000135,Работы строительные специализированные,43.9,1898111.58,122,"[работы, строительные, специализированные]","['работ', 'строительн', 'специализирова']","[работы, строительные, специализированные]","[работы, строительные, специализированные]","[работы, строительные, специализированные]"
3,137827,2590500335021000676,Работы строительные специализированные,43.3,341914.00,199,"[работы, строительные, специализированные]","['работ', 'строительн', 'специализирова']","[работы, строительные, специализированные]","[работы, строительные, специализированные]","[работы, строительные, специализированные]"
4,138113,2590615606021000060,Услуги в области архитектуры и инженерно-техни...,71.1,1714352.37,383,"[услуги, в, области, архитектуры, и, инженерно...","['услуг', 'област', 'архитектур', 'проектирова...","[услуги, в, области, архитектуры, и, инженерно...","[услуги, в, области, архитектуры, и, инженерно...","[услуги, области, архитектуры, инженерно-техни..."


In [ ]:
df_save = df[['id',	'contract_number',	'object_name',	'object_code',	'cost',	'contract_execution_days',	'data',	'data_final']]

df_save .to_csv('datasets/dataset_nlp.csv', sep=';', index=False)

In [57]:
df = df.dropna()

data = df['data'], 
y = df["object_code"], 
data_final = df[['data_final']]
data_final

,data_final
0,"['работ', 'строительн', 'специализирова']"
1,"['работ', 'строительн', 'специализирова']"
2,"['работ', 'строительн', 'специализирова']"
3,"['работ', 'строительн', 'специализирова']"
4,"['услуг', 'област', 'архитектур', 'проектирова..."
...,...
551495,"['работ', 'строительн', 'специализирова']"
551496,"['работ', 'строительн', 'специализирова']"
551497,"['работ', 'строительн', 'специализирова']"
551498,"['услуг', 'област', 'архитектур', 'проектирова..."


In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import precision_score, f1_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB

In [68]:
bagwords = CountVectorizer(max_features=1500,  min_df=1, max_df=1, stop_words=stopwords.words('russian')) 
bagwords_X = bagwords.fit_transform(data_final).toarray()
bagwords_X 

array([[1]], dtype=int64)

In [70]:
bagwords_X_train, bagwords_X_test, bagwords_y_train, bagwords_y_test = train_test_split(bagwords_X,
                                                                                        y,
                                                                                        test_size=0.2,
                                                                                        random_state=0)

ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [30]:
naive_bayes = MultinomialNB()
bagwords_nb = naive_bayes.fit(bagwords_X_train, bagwords_y_train)
bagwords_nb_predictions = bagwords_nb.predict(bagwords_X_test)

bagwords_nb_f1 = f1_score(bagwords_y_test, bagwords_nb_predictions, average='weighted')
bagwords_nb_precision = precision_score(bagwords_y_test, bagwords_nb_predictions, average='weighted')
print(f'F-мера модели "Мешок слов": {round(bagwords_nb_f1, 3)}, точность: {round(bagwords_nb_precision, 3)}')

NameError: name 'bagwords_X_train' is not defined

In [ ]:

tfidf = TfidfVectorizer(max_features=1700, min_df=5, max_df=0.7, stop_words=stopwords.words('russian'))
tfidf_X = tfidf.fit_transform(data_final).toarray()
tfidf_X_train, tfidf_X_test, tfidf_y_train, tfidf_y_test = train_test_split(tfidf_X,
                                                                            y,
                                                                            test_size=0.2,
                                                                            random_state=0)
tfidf_nb = naive_bayes.fit(tfidf_X_train, tfidf_y_train)
tfidf_nb_predictions = tfidf_nb.predict(tfidf_X_test)
tfidf_nb_f1 = f1_score(tfidf_y_test, tfidf_nb_predictions, average='weighted')
tfidf_nb_precision = precision_score(tfidf_y_test, tfidf_nb_predictions, average='weighted')
print(f'F-мера модели TF-IDF: {round(tfidf_nb_f1, 3)}, точность: {round(tfidf_nb_precision, 3)}')

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=1, max_iter=10)
tfidf_nb = model.fit(tfidf_X_train, tfidf_y_train)
tfidf_nb_predictions = model.predict(tfidf_X_test)
tfidf_nb_f1 = f1_score(tfidf_y_test, tfidf_nb_predictions, average='weighted')
tfidf_nb_precision = precision_score(tfidf_y_test, tfidf_nb_predictions, average='weighted')
print(f'F-мера модели TF-IDF: {round(tfidf_nb_f1, 3)}, точность: {round(tfidf_nb_precision, 3)}')